# 环境介绍
本示例运行在colap ，使用A100运行

# 本示例概要
本示例主要是带大家安装llava ,并透过llava实现进行图片解读



1️⃣ 檢查 GPU 是否可用
在 Colab 內執行以下命令，確保啟用了 GPU（建議 A100 或更強的顯卡）：

In [ ]:
!nvidia-smi

2️⃣ 安裝所需的依賴庫
執行以下命令來安裝 LLaVA 需要的套件：

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/diffusers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/huggingface_hub.git


3️⃣ 下載 LLaVA 專案 并使用safetensors这种高效的权重格式来减少记忆体占用,这边要注意要先建立LLaVA档案夹
这边要注意的是我clone llava模型地址会变动，可能会因为不同版本导致安装过程有变化哦！

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
%cd LLaVA
!pip install -e .
!pip install safetensors

因为避免OOM，我这里手動設定 PyTorch CUDA 記憶體管理512来减少GPU记忆体碎片化提高可用记忆体

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"


因为安裝 bitsandbytes（8-bit 量化庫）（大幅降低顯存占用），在环境的设定上指定统一格式

In [ ]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install bitsandbytes


4️⃣ 下載 LLaVA 預訓練模型
这边的优化项目我一一说明

🔹 BitsAndBytesConfig 的作用

1.   load_in_8bit=True（啟用 8-bit 量化，降低 VRAM 需求）
這行表示：將 LLaVA 模型的權重轉換為 8-bit 精度（預設情況下是 16-bit fp16 或 32-bit fp32）。這樣可以大幅減少模型的 GPU 記憶體占用（約降低 50%），但仍保留一定的計算精度。🔹 效益：降低 VRAM 需求，使模型能夠在 Colab A100上運行。
比 4-bit 更準確，但比 16-bit 省記憶體。
2.   llm_int8_enable_fp32_cpu_offload=True（將 FP32 權重卸載到 CPU，釋放 VRAM）
這行表示：部分需要高精度的 FP32 權重將被卸載到 CPU，減少 GPU 記憶體使用。
在 8-bit 量化中，某些特定的層仍然需要 fp32 來保持模型的運算穩定性，所以這裡讓它們移到 CPU 來節省 GPU 記憶體。🔹 效益：避免「CUDA Out of Memory (OOM)」錯誤。
減少記憶體碎片化，提高運行效率。
3.   load_in_8bit_fp32_cpu_offload=True（進一步將 FP32 權重完全移到 CPU）
這行表示：完全將 FP32 權重存放在 CPU，而不是 GPU。這可以進一步降低 GPU 記憶體需求，但可能會降低推理速度（因為需要 CPU-GPU 數據傳輸）。🔹 效益：減少 GPU 記憶體占用，確保 LLaVA 可以在低記憶體環境下運行。
4.   bnb_4bit_compute_dtype=torch.bfloat16（計算時使用 bfloat16 以提升精度）
這行表示：設置 4-bit 計算的數據類型為 bfloat16（BF16）。bfloat16 是一種較新的數據格式，能夠保持 fp32 精度的範圍，但只占一半記憶體。🔹 效益：提高數值計算的穩定性，避免過度量化導致的精度損失。相比 fp16，bfloat16 更穩定且更適合較新的 GPU（如 A100、V100）。
5.   bnb_4bit_quant_type="nf4"（使用 NF4 量化，提高 4-bit 模型的精度）
這行表示：使用 NF4（Normal Float 4）來做 4-bit 量化。
NF4 是 bitsandbytes 特有的一種 4-bit 量化格式，它比傳統的 4-bit 量化方法更準確。
🔹 效益：能夠在 4-bit 量化的情況下保留更高的精度。避免過度量化造成模型性能下降。

🔹 load_pretrained_model 的作用


1.   device_map="auto"（自動選擇 CPU 或 GPU 運行）讓 transformers 自動分配模型到 GPU 或 CPU，確保不會超過 GPU 記憶體。若 VRAM 不足，會自動將部分層卸載到 CPU。
🔹 效益：防止 CUDA 記憶體不足（OOM）錯誤。根據硬體資源自動最佳化。
2.   offload_folder="./content/offload_weights"（存放卸載的權重）當 VRAM 不夠時，某些層會被卸載到磁碟，這裡指定它們存放的位置。🔹 效益：允許 LLaVA 在低記憶體環境運行（Colab T4、V100）。避免記憶體不足導致崩潰。
3.   quantization_config=quantization_config（啟用 8-bit 量化配置）告訴 load_pretrained_model 使用 剛剛定義的 8-bit 量化配置。🔹 效益：
讓 LLaVA 以更少的 GPU 記憶體運行。確保 8-bit 量化和 CPU Offload 設置生效












In [ ]:
import torch
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from transformers import BitsAndBytesConfig, AutoTokenizer

model_path = "liuhaotian/llava-v1.5-7b"

# Try loading in 8-bit with CPU offload and fp32 cpu offload
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Load in 8-bit
    llm_int8_enable_fp32_cpu_offload=True,  # Enable CPU offload for fp32 weights
    #load_in_8bit_fp32_cpu_offload=True,  # Offload fp32 weights to CPU
    bnb_4bit_compute_dtype=torch.bfloat16,  # Set compute dtype for 4-bit quantization (if needed)
    bnb_4bit_quant_type="nf4"  # Set quantization type for 4-bit quantization (if needed)
)
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    device_map="auto",                # Use automatic device mapping
    offload_folder="./content/offload_weights", # Set the offload folder
    quantization_config=quantization_config  # Pass the quantization config
)

5️⃣ 測試 LLaVA
上传一张图片请llava根据我的prompt指令进行描述

In [ ]:
from llava.eval.run_llava import eval_model

prompt = "這張圖片中有什麼值得注意的地方？"
image_file = "https://llava-vl.github.io/static/images/view.jpg"

args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)


You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


這張圖片中有一個漂亮的木板步道，它穿過一個湖，並且有一個山脈在背景中。這個步道的設計非常吸引人，因為它從湖面上延伸到山脈上，讓人感受到自然的美麗和寧靜。此外，這個步道的位置非常優越，因為它可以讓人欣賞到湖面和山脈的美景，並且可以提供一個漫步或遠足的好地方。
